## 大課題 毒キノコの判定 SVM
https://diver.diveintocode.jp/textbooks/568
###### 流れ:  
・導入
・データの取得  
・前処理  
・ハイパーパラメータの調整  
・SVMの実行  

### 1. 【課題】SVMとは何か
##### ・どのようなときに使うのか
###### 答え:
分類と回帰の両方を扱う問題で汎化能力を高めるモデルを構築したいときに使う。  
また、未学習のデータに対しても高い識別性能を発揮する。   
扱う特徴量としては、高次元の学習に向いている手法であり、データの個数にも制限はない。  
しかし、場合によってはサンプリングすることが必要になる。  

参考文献:   
https://docs.oracle.com/cd/B28359_01/datamine.111/b28129/algo_svm.htm#DMCON025  
http://home.hiroshima-u.ac.jp/tkurita/lecture/svm.pdf

##### ・どのような仕組みなのか
###### 答え:
異なるクラスに属しながら、お互いの距離に一番近い位置に存在する特徴量同士をサポートベクターと呼び、その特徴量を使って構築するモデル。  
それぞれのサポートベクター(クラス間では一番近い)から一番離れている距離(つまり、お互いの中間距離)を、データを分類するための決定境界の位置とする。  
この決定境界を決めることを、マージンの最大化と呼び、汎化能力の指標となる。  
##### ・どういった利点欠点があるのか
###### 答え:
利点としては、機械学習モデルの中では、汎化能力の高いモデルである。  
欠点としては、お互いの特徴量が同一のスケール(例えば、0から1の範囲)に収まるように、注意深く前処理をする必要がある。

### 2. 必要なライブラリをimport

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing as spp
from sklearn.model_selection import train_test_split
%matplotlib inline

### 3. 【課題】データを取得する

In [2]:
set_names = ['class','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing',
             'gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring',
             'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color',
             'ring-number','ring-typ','spore-print-color','poplation','habitat']
df = pd.read_csv("agaricus-lepiota.data.txt", names = set_names)
df.head(10)

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-typ,spore-print-color,poplation,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


### 4.【課題】データの変換

In [3]:
df_scaled = df
for column in set_names:
    le = spp.LabelEncoder()
    le.fit(df[column])
    df_scaled[column] = le.transform(df_scaled[column])
df_scaled.head(10)

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-typ,spore-print-color,poplation,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
5,0,5,3,9,1,0,1,0,0,5,...,2,7,7,0,2,1,4,2,2,1
6,0,0,2,8,1,0,1,0,0,2,...,2,7,7,0,2,1,4,2,2,3
7,0,0,3,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,3,3
8,1,5,3,8,1,6,1,0,1,7,...,2,7,7,0,2,1,4,2,4,1
9,0,0,2,9,1,0,1,0,0,2,...,2,7,7,0,2,1,4,2,3,3


### 5.【課題】データセットの分割

In [4]:
X = df.iloc[:, 1:] # 全ての行(「:,」の部分)と１列目から最後の列(「1:」の部分)まで取得
y = df.iloc[:, :1] # 全ての行(「:,」の部分)と先頭から1列目(「:1」の部分)まで取得

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-typ,spore-print-color,poplation,habitat
7434,3,2,3,0,5,1,1,0,7,0,...,2,7,7,0,2,2,4,7,3,1
7725,5,0,8,0,5,1,1,0,2,0,...,2,7,7,0,2,2,4,7,2,1
783,5,2,8,1,3,1,0,0,5,0,...,2,7,7,0,2,1,4,2,3,3
1928,2,2,8,0,5,1,1,0,3,1,...,0,7,7,0,2,1,0,2,3,1
7466,3,3,2,0,8,1,0,1,0,1,...,1,7,6,0,2,1,0,7,4,2


### 6.【課題】標準化

In [5]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer

s_scaler = StandardScaler()
s_scaler.fit(X_train)

r_scaler = RobustScaler()
r_scaler.fit(X_train)

mm_scaler = MinMaxScaler()
mm_scaler.fit(X_train)

print("StandardScaler: ", s_scaler.transform(X_train))
print("RobustScaler: ", r_scaler.transform(X_train))
print("MinMaxScaler: ", mm_scaler.transform(X_train))

StandardScaler:  [[-0.2134376   0.14364556 -0.58042016 ...,  1.42385604 -0.5040274
  -0.29674863]
 [ 1.03510511 -1.47851339  1.3980497  ...,  1.42385604 -1.29620976
  -0.29674863]
 [ 1.03510511  0.14364556  1.3980497  ..., -0.6737141  -0.5040274
   0.86329309]
 ..., 
 [ 1.03510511  0.14364556 -0.58042016 ..., -0.25420007 -0.5040274
  -0.29674863]
 [-0.83770895 -1.47851339 -0.18472619 ..., -0.25420007  0.28815496
  -0.8767695 ]
 [ 1.03510511  0.95472503 -0.58042016 ..., -0.25420007  1.08033733
  -0.8767695 ]]
RobustScaler:  [[ 0.          0.         -0.2        ...,  0.8        -1.          0.        ]
 [ 0.66666667 -0.66666667  0.8        ...,  0.8        -2.          0.        ]
 [ 0.66666667  0.          0.8        ..., -0.2        -1.          1.        ]
 ..., 
 [ 0.66666667  0.         -0.2        ...,  0.         -1.          0.        ]
 [-0.33333333 -0.66666667  0.         ...,  0.          0.         -0.5       ]
 [ 0.66666667  0.33333333 -0.2        ...,  0.          1.      

In [6]:
# scalerを変えて実行
# X_train_scaled = s_scaler.transform(X_train)
# X_test_scaled = s_scaler.transform(X_test)

# X_train_scaled = r_scaler.transform(X_train)
# X_test_scaled = r_scaler.transform(X_test)

X_train_scaled = mm_scaler.transform(X_train)
X_test_scaled = mm_scaler.transform(X_test)

##### ・SVMを行う際の標準化の必要性
###### 答え:
値の大きい特徴量に、値の小さい特徴量が引きずられ、正しい予測ができなくなる可能性があるため。  
(逆に、値の大きい特徴量が、値の小さい特徴量に引きずられることもある。)  
また、外れ値に引きずられ、過学習した予測を行ってしまう可能性があるため。  
SVMでのモデル予測を行うために使うカーネル関数は、特徴量同士のベクトルの内積を行う。  
SVMでのベクトルの内積計算の元となるベクトルは、φ関数を使った特徴量同士の「素の合計」である。    
値の大きい特徴量は、「素」の数も多くなる可能性があり、ベクトル数も多くなってしまう。  
それによりベクトル数の少ない特徴量が、ベクトル数の多い特徴量に合わせる写像となってしまう。  
その結果、上述の「正しくない予測」や「過学習」を引き起こしてしまう。  

参考:  
https://diver.diveintocode.jp/textbooks/302  
https://diver.diveintocode.jp/textbooks/566

### 7. 【課題】カーネルとは何か
###### 答え:
SVMにおいては、単純な線形のモデルでは分類できない複雑な分類を行う際に、特徴量を高次元に移す写像を行うことを指す。  
特徴量の「素」の数を用いて内積計算をするカーネル関数をカーネルトリックと呼ぶ。

### 8.【課題】コストペナルティCとは何か
###### 答え:
SVMにおいて、誤分類をどれほど許容するかを決めるパラメータ。  
値が小さいほど誤分類を許容し、値が大きいほど誤分類を許容しない。  

### 9. ハイパーパラメータを調整する

In [7]:
X_train_scaled_grid, X_test_scaled_gridval, y_train_grid, y_test_gridval = train_test_split(X_train_scaled, y_train, test_size = 0.2, random_state = 0)

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_candidates = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.01, 0.001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.01, 0.001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.01, 0.001]}
    ]
clf = GridSearchCV(SVC(), param_candidates)
clf.fit(X_train_scaled_grid, y_train_grid.values.ravel())

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']}, {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.01, 0.001]}, {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.01, 0.001]}, {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.01, 0.001]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [9]:
clf.grid_scores_

/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.94268, std: 0.00510, params: {'C': 1, 'kernel': 'linear'},
 mean: 0.96961, std: 0.00807, params: {'C': 10, 'kernel': 'linear'},
 mean: 0.98192, std: 0.00785, params: {'C': 100, 'kernel': 'linear'},
 mean: 0.98596, std: 0.00452, params: {'C': 1000, 'kernel': 'linear'},
 mean: 0.93210, std: 0.00555, params: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.88690, std: 0.00452, params: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.94711, std: 0.00510, params: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.93153, std: 0.00592, params: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.97730, std: 0.00661, params: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.94614, std: 0.00440, params: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.99962, std: 0.00054, params: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.94749, std: 0.00452, params: {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.92460, std: 0.00507, params: {'C': 1, 'gamma': 0.01, 'ker

In [10]:
clf.best_params_

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [11]:
clf.score(X_test_scaled_gridval, y_test_gridval)

1.0

### 10. 【課題】学習およびテスト

In [12]:
svm_rbf = SVC(kernel = 'rbf', random_state = 0, gamma = 0.01, C = 1000)
svm_rbf.fit(X_train_scaled, y_train.values.ravel())
svm_rbf.score(X_test_scaled, y_test)

1.0

### ・StandardScalerの結果
best_params_: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}  
グリッドサーチのscore: 1.0  
サポートベクタマシンのscore(ベストパラメータのkernelを使用): 1.0 

### ・RobustScalerの結果
best_params_: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}  
グリッドサーチのscore: 1.0  
サポートベクタマシンのscore(ベストパラメータのkernelを使用): 1.0

### ・MinMaxScalerの結果
best_params_: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}  
グリッドサーチのscore: 1.0  
サポートベクタマシンのscore(ベストパラメータのkernelを使用): 1.0

##### 最適なパラメータは、全て同じものになりました。